In [ ]:
!pip install sktime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
os.chdir("drive/MyDrive/LXY")

In [2]:
from torch.serialization import load
import torch
import numpy as np
import sktime
from networks import *

from torch import nn
import torch.utils.data as Data
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
from sklearn.preprocessing import LabelEncoder, StandardScaler
#from data_loader import load_from_tsfile_to_dataframe
from sktime.datasets import load_from_tsfile_to_dataframe

In [96]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = 'UWaveGestureLibrary'
batch_size = 32
seq_length = 315
feature_size = 3
num_hiddens = 64
num_heads = 8
num_layers = 1
num_cat = 8
dropout = 0.1
mask_prob = 0.3
kernel_size = 20
learning_rate = 1e-3

In [37]:
le = LabelEncoder()
scaler = StandardScaler()

load train and test data

In [38]:
data_path = f'Multivariate_ts/{dataset}/{dataset}_TRAIN.ts'
train_X, train_Y = load_from_tsfile_to_dataframe(data_path)
train_X = from_nested_to_3d_numpy(train_X)
train_X = train_X.transpose(0, 2, 1)
#replace nan with 0
train_X[np.isnan(train_X)] = 0

In [39]:
data_path = f'Multivariate_ts/{dataset}/{dataset}_TEST.ts'
test_X, test_Y = load_from_tsfile_to_dataframe(data_path)
test_X = from_nested_to_3d_numpy(test_X)
test_X = test_X.transpose(0, 2, 1)
#x_3d = torch.tensor(from_nested_to_3d_numpy(test_x), dtype=torch.float32, device=device)
#inplace nan with 0
test_X[np.isnan(test_X)] = 0

preprocess

In [40]:
train_X = train_X.reshape(-1, feature_size)
test_X = test_X.reshape(-1, feature_size)
scaler.fit(train_X)

train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)
train_X = train_X.reshape(-1, seq_length, feature_size)
test_X = test_X.reshape(-1, seq_length, feature_size)
train_X = torch.tensor(train_X, dtype=torch.float32, device=device)
test_X = torch.tensor(test_X, dtype=torch.float32, device=device)

### for classification 
le.fit(train_Y)
train_Y = le.transform(train_Y)
test_Y = le.transform(test_Y)
train_Y = torch.tensor(train_Y, device=device)
test_Y = torch.tensor(test_Y, device=device)

### for regression
#train_Y = torch.tensor(train_Y, dtype=torch.float32, device=device)
#test_Y = torch.tensor(test_Y, dtype=torch.float32, device=device)

load data into dataloader

In [41]:
train_dataset = Data.TensorDataset(train_X, train_Y)
train_loader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)
#del train_X

In [42]:
test_dataset = Data.TensorDataset(test_X, test_Y)
test_loader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = True
)

Pretrain

In [48]:
pre_model = TimeTransformer(seq_length, feature_size, num_hiddens, 
            num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_hiddens, 
            num_hiddens*2, num_heads, num_layers, dropout, mask_prob).to(device)
optimizer = torch.optim.Adam(pre_model.parameters(), lr=learning_rate)
loss_fn = MaskedWeightedMSELoss()
#pre_model.load_state_dict(torch.load(f'parameters/{dataset}_pretrained.pt'))

In [ ]:
for epoch in range(100):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        recover_x = pre_model(batch_x, batch_x)
        train_loss = loss_fn(recover_x, batch_x, pre_model.mask)
        epoch_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()
    if epoch%10 == 9:
        print(f"epoch {epoch+1}, loss = {epoch_loss}")
torch.save(pre_model.state_dict(), f'parameters/{dataset}_pretrained.pt')

Finetune

In [50]:
model = TimeTransformerClassifier(seq_length, feature_size, num_hiddens, 
            num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_hiddens, 
            num_hiddens*2, num_heads, num_layers, dropout, num_cat).to(device)
model.encoder = pre_model.encoder
lossfn = nn.CrossEntropyLoss()
#model.load_state_dict(torch.load(f'parameters/{dataset}_finetune.pt'))
#lossfn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(50):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        #y_pred = model(batch_x)
        y_pred = model(batch_x).squeeze()
        train_loss = lossfn(y_pred, batch_y)
        epoch_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()
    if epoch%10 == 9:
        print(f"epoch {epoch+1}, loss = {epoch_loss}")
torch.save(model.state_dict(), f'parameters/{dataset}_finetune.pt')
#del pre_model

In [ ]:
'''
with torch.no_grad():
    y_h = []
    y_t = []
    for (batch_x, batch_y) in test_loader:
        pred = model(batch_x)
        y_h.append(pred)
        y_t.append(batch_y)
    y_h = torch.cat(y_h).squeeze()
    y_t = torch.cat(y_t).squeeze()
    loss = torch.sqrt(lossfn(y_h, y_t)).item()
loss
'''

In [55]:
# classify
model.eval()
with torch.no_grad():
    pred = model(test_X).squeeze()
    accuracy = (pred.argmax(1) == test_Y).sum()/test_X.shape[0]
accuracy

tensor(0.8094, device='cuda:0')

In [ ]:
# regression
model.eval()
with torch.no_grad():
    pred = model(test_X).squeeze()
    loss = torch.sqrt(lossfn(pred, test_Y)).item()
loss

Supervised Only

In [56]:
del model
torch.cuda.empty_cache()
model = TimeTransformerClassifier(seq_length, feature_size, num_hiddens, 
            num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_hiddens, 
            num_hiddens*2, num_heads, num_layers, dropout, num_cat).to(device)
lossfn = nn.CrossEntropyLoss()
#lossfn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(100):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        #y_pred = model(batch_x)
        y_pred = model(batch_x).squeeze()
        train_loss = lossfn(y_pred, batch_y)
        epoch_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()
    if epoch%10 == 9:
        print(f"epoch {epoch+1}, loss = {epoch_loss}")
torch.save(model.state_dict(), f'parameters/{dataset}_transformer.pt')

In [ ]:
# classify
model.eval()
with torch.no_grad():
    pred = model(test_X).squeeze()
    accuracy = (pred.argmax(1) == test_Y).sum()/test_Y.shape[0]
accuracy

In [ ]:
# regression
with torch.no_grad():
    pred = model(test_X).squeeze()
    loss = torch.sqrt(lossfn(pred, test_Y)).item()
loss

Dual Transformer

In [100]:
del model
torch.cuda.empty_cache()
model = DualClassifier(seq_length, feature_size, num_hiddens, 
            num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_hiddens, 
            num_hiddens*2, num_heads, num_layers, dropout, kernel_size,
            num_cat).to(device)
model.train()
lossfn = nn.CrossEntropyLoss()
#lossfn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(40):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        #y_pred = model(batch_x)
        y_pred = model(batch_x).squeeze()
        train_loss = lossfn(y_pred, batch_y)
        epoch_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()
    if epoch%10 == 9:
        print(f"epoch {epoch+1}, loss = {epoch_loss}")
torch.save(model.state_dict(), f'parameters/{dataset}_dual_transformer.pt')

In [ ]:
# classify
model.eval()
with torch.no_grad():
    pred = model(test_X).squeeze()
    accuracy = (pred.argmax(1) == test_Y).sum()/test_Y.shape[0]
accuracy

In [ ]:
with torch.no_grad():
    pred = model(test_X)[:, -1, :].squeeze()
loss = torch.sqrt(lossfn(pred, test_Y)).item()
loss

LSTM

In [80]:
class RNNModel(nn.Module):
    def __init__(self, seq_length, num_features, num_hiddens, num_layers, num_cats):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(num_features, num_hiddens, num_layers, batch_first=True)
        self.Linear = nn.Linear(num_hiddens, num_cats)

    def forward(self, x):
        out, (hn, cn) = self.rnn(x)
        out = torch.squeeze(out[:, -1, :])
        return self.Linear(out)

In [87]:
del model
torch.cuda.empty_cache()
model = RNNModel(seq_length, feature_size, num_hiddens, 3*num_layers, num_cat).to(device)
lossfn = nn.CrossEntropyLoss()
#lossfn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(50):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(batch_x).squeeze()
        train_loss = lossfn(y_pred, batch_y)
        epoch_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()
    if epoch%10 == 9:
        print(f"epoch {epoch+1}, loss = {epoch_loss}")
torch.save(model.state_dict(), f'parameters/{dataset}_RNN.pt')

In [89]:
with torch.no_grad():
    pred = model(test_X).squeeze()
    accuracy = (pred.argmax(1) == test_Y).sum()/test_Y.shape[0]
accuracy

tensor(0.4656, device='cuda:0')

In [ ]:
with torch.no_grad():
    pred = model(test_X).squeeze()
loss = torch.sqrt(lossfn(pred, test_Y)).item()
loss